In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\Admin\Data\multimodal-rag-baseline


In [2]:
from configs.config import *
from libs.common import *
load_dotenv(find_dotenv())

c:/Users/Admin/Data/multimodal-rag-baseline


c:\Users\Admin\miniconda3\envs\kyanon_digital\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
import os

dataset = load_dataset("hwyin04/table-cross-page")

save_dir = f"{data_dir}/pdf"
os.makedirs(save_dir, exist_ok=True)

for split in dataset.keys():
    path = os.path.join(save_dir, f"{split}.jsonl")
    dataset[split].to_json(path, orient="records", lines=True)
    print(f"Saved {split} to {path}")


To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 165.55ba/s]

Saved train to {data_dir}/pdf\train.jsonl


In [4]:

def download_pdfs_from_jsonl(jsonl_file):
    with open(jsonl_file, 'r') as f:
        for line in f:
            data = json.loads(line.strip())
            
            pdf_data = data.get('pdf')
            if pdf_data and 'path' in pdf_data:
                pdf_path = pdf_data['path']

                if pdf_path:
                    try:
                        shutil.copy(pdf_path, save_dir)
                        print(f"Đã tải: {pdf_path}")
                    except Exception as e:
                        print(f"Không thể tải {pdf_path}: {e}")
            else:
                print(f"Không tìm thấy khóa 'path' trong dữ liệu: {data}")

download_pdfs_from_jsonl(f'{save_dir}/train.jsonl')


Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\0c92f65db928c431023f59603039aa1e.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\0ed7f2ecfd607a42c745b0889e500ecf.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\0f2eac764c8d04d8d13d8a999342d106.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\2c98e99a08ec5392d50e60370d871319.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\2d64938a8d3e12dbb709f760ecd53e14.pdf
Đã tải: C:\Users\Admin\.cache\huggingface\hub\datasets--hwyin04--table-cross-page\snapshots\9e687f369e1ab5282060a543c831ad91449d66c3\2e1b34aa28e718f66